# 데이콘: 이커머스 고객 세분화 분석 아이디어 경진대회
---

- 해당 대회에 대한 간단한 소개와 인사글은 README.md 파일에 작성되어 있습니다.

- 혹시 보고싶으신 분은 [이 텍스트를 클릭해주세요. 깃허브 링크로 연결됩니다](https://github.com/Randy-archive/ecommerce_seg)

---

안녕하세요, 대회에 참여하게 된 랜디Randy입니다.

RFM이나 Cohort분석에 대해서 이번 대회를 통해 더 깊게 이해해보고자 합니다.

목차는 아래와 같습니다.

# [Index]
---

1. 데이터 살펴보기

2. 오프라인 마케팅 비용이 같은 기간동안 온라인 마케팅 비용 증감에 따른 매출액 변화


In [13]:
# 필요한 라이브러리 불러오기
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 한글 폰트 설정하기
import matplotlib.font_manager as fm
fe = fm.FontEntry(fname = 'MaruBuri-Regular.otf', name = 'MaruBuri')
fm.fontManager.ttflist.insert(0, fe)
plt.rc('font', family='MaruBuri')

# [1. 데이터 살펴보기]

어떤 데이터가 어떤 형식으로 존재하는지 한번 살펴보도록 하겠습니다.

In [14]:
customer_data = pd.read_csv('Customer_info.csv')
discount_data = pd.read_csv('Discount_info.csv')
marketing_data = pd.read_csv('Marketing_info.csv')
onlinesales_data = pd.read_csv('Onlinesales_info.csv')
tax_data = pd.read_csv('Tax_info.csv')

customer = customer_data.copy()
discount = discount_data.copy()
marketing = marketing_data.copy()
onlinesales = onlinesales_data.copy()
tax = tax_data.copy()

print(customer.shape)
customer.head()
# 고객 ID, 성별, 고객지역, 가입기간(단위: 월)

(1468, 4)


,고객ID,성별,고객지역,가입기간
0,USER_1358,남,Chicago,12
1,USER_0190,남,California,43
2,USER_0066,남,Chicago,33
3,USER_0345,여,California,30
4,USER_0683,남,California,49


In [15]:
print(discount.shape)
discount.head()
# 월(month), 제품카테고리, 쿠폰코드, 할인율(%)

(204, 4)


,월,제품카테고리,쿠폰코드,할인율
0,Jan,Apparel,SALE10,10
1,Feb,Apparel,SALE20,20
2,Mar,Apparel,SALE30,30
3,Jan,Nest-USA,ELEC10,10
4,Feb,Nest-USA,ELEC20,20


In [20]:
print(marketing.shape)
marketing.head()
# 날짜, 오프라인 마케팅으로 지출한 비용(달러), 온라인 마케팅으로 지출한 비용(달러)

(365, 3)


,날짜,오프라인비용,온라인비용
0,2019-01-01,4500,2424.50
1,2019-01-02,4500,3480.36
2,2019-01-03,4500,1576.38
3,2019-01-04,4500,2928.55
4,2019-01-05,4500,4055.30


In [17]:
print(onlinesales.shape)
onlinesales.head()
# 고객 고유 ID, 거래 고유 ID, 거래 날짜, 제품 고유 ID, 제품 카테고리, 주문수량,
# 평균금액 (동일 상품이어도 세부 옵션에 따라 가격 상이, 단위 달러)
# 배송료(단위 달러), 쿠폰 적용 상태

(52924, 9)


,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태
0,USER_1358,Transaction_0000,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used
1,USER_1358,Transaction_0001,2019-01-01,Product_0981,Nest-USA,1,153.71,6.5,Used
2,USER_1358,Transaction_0002,2019-01-01,Product_0904,Office,1,2.05,6.5,Used
3,USER_1358,Transaction_0003,2019-01-01,Product_0203,Apparel,5,17.53,6.5,Not Used
4,USER_1358,Transaction_0003,2019-01-01,Product_0848,Bags,1,16.50,6.5,Used


In [18]:
print(tax.shape)
tax.head()
# 제품 카테고리, Goods and Services Tax(%)

(20, 2)


,제품카테고리,GST
0,Nest-USA,0.10
1,Office,0.10
2,Apparel,0.18
3,Bags,0.18
4,Drinkware,0.18


# [2. 오프라인 마케팅 비용이 같은 기간동안 온라인 마케팅 비용 증감에 따른 매출액 변화]
---

`Marketing_info.csv`를 보면 날짜, 오프라인비용, 온라인 비용이 존재합니다.

온라인 비용은 날마다 바뀌지만, 오프라인 비용은 일정 기간동안 동일하다는 것을 발견했습니다.

따라서 오프라인 비용이 동일할 때, 온라인 비용이 증가하고 감소함에 따라 매출액에는 어떤 변화가 있는지 살펴보도록 하겠습니다.

In [95]:
classified_bills = []
'''
오프라인 비용이 똑같은 기간을 하나의 DataFrame으로 구성해서,
classified_bills 리스트에 각각의 DataFrame으로 저장하려고 합니다.
예를 들어 오프라인 비용이 4000달러가 고정인 채로 일주일이 지속된다면
일주일 동안의 데이터는 모두 하나의 DataFrame에 저장되고,

다음 날부터 오프라인 비용이 4500달러가 된다면
새로운 DataFrame에 오프라인비용이 4500인 행들이 쌓이게 됩니다.

날짜가 연속적일때만 하나의 DataFrame으로 삽입되며,
오프라인비용이 같다 할지라도 날짜가 연속적이지 않을 경우
별도의 DataFrame으로 구성합니다.
'''

standard = 0
tmp = pd.DataFrame()
for idx, row in marketing.iterrows():

    if row['오프라인비용'] == marketing.loc[standard, '오프라인비용']:
        tmp = pd.concat([tmp, pd.DataFrame([row])], ignore_index=True)
    else:
        standard = idx
        classified_bills.append(tmp)
        tmp = pd.DataFrame()
        tmp = pd.concat([tmp, pd.DataFrame([row])], ignore_index=True)

classified_bills

[           날짜  오프라인비용    온라인비용
 0  2019-01-01    4500  2424.50
 1  2019-01-02    4500  3480.36
 2  2019-01-03    4500  1576.38
 3  2019-01-04    4500  2928.55
 4  2019-01-05    4500  4055.30
 5  2019-01-06    4500  3796.85
 6  2019-01-07    4500  2579.52,
            날짜  오프라인비용    온라인비용
 0  2019-01-08    5000  2551.38
 1  2019-01-09    5000  3287.83
 2  2019-01-10    5000   515.44
 3  2019-01-11    5000  1757.58
 4  2019-01-12    5000  2435.97
 5  2019-01-13    5000  2449.65
 6  2019-01-14    5000  1920.57,
            날짜  오프라인비용    온라인비용
 0  2019-01-15    3000  1734.68
 1  2019-01-16    3000  3727.61
 2  2019-01-17    3000   417.73
 3  2019-01-18    3000  1312.50
 4  2019-01-19    3000  2753.56
 5  2019-01-20    3000  2073.51
 6  2019-01-21    3000   640.93,
            날짜  오프라인비용    온라인비용
 0  2019-01-22    1000  1049.73
 1  2019-01-23    1000  1356.59
 2  2019-01-24    1000   560.02
 3  2019-01-25    1000  1124.95
 4  2019-01-26    1000   937.58
 5  2019-01-27    1000   687.86
 6  2